In [5]:
!pip install dash jupyter-dash plotly -q


In [6]:
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px

# Sample df (replace with your df later)
df = pd.DataFrame({
    "Launch Site": ["A", "B", "A", "C"],
    "Payload Mass (kg)": [2000, 5000, 9000, 3000],
    "class": [1, 0, 1, 0],
    "Booster Version Category": ["v1", "v2", "v1", "v3"]
})

# Create app
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Test Dash App in Colab"),

    dcc.Graph(
        id="test-graph",
        figure=px.scatter(df, x="Payload Mass (kg)", y="class")
    )
])

# Run IN COLAB
app.run_server(mode='inline')


/usr/local/lib/python3.12/dist-packages/dash/dash.py:642: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



AttributeError: 'super' object has no attribute 'run_server'

In [7]:
!pip install dash plotly -q

import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from google.colab import output
import threading
import time

# ----------------------------
#  SAMPLE DATA (Guaranteed working)
#  Replace this later with your real df
# ----------------------------

df = pd.DataFrame({
    "Launch Site": ["CCAFSC LC-40", "VAFB SLC-4E", "KSC LC-39A", "CCAFS LC-40", "KSC LC-39A"],
    "Payload Mass (kg)": [3000, 6500, 12000, 5000, 11000],
    "class": [1, 0, 1, 1, 0],
    "Booster Version Category": ["v1.0", "v1.1", "FT", "v1.1", "FT"]
})

# ----------------------------
# DASH APP
# ----------------------------

app = Dash(__name__)

app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'All Sites'}] +
                [{'label': s, 'value': s} for s in df['Launch Site'].unique()],
        value='All Sites'
    ),

    html.Br(),

    dcc.Graph(id='success-pie-chart'),

    html.Br(),

    html.P("Payload Range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=df['Payload Mass (kg)'].min(),
        max=df['Payload Mass (kg)'].max(),
        step=500,
        value=[df['Payload Mass (kg)'].min(), df['Payload Mass (kg)'].max()]
    ),

    html.Br(),

    dcc.Graph(id='success-payload-scatter-chart')
])


@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie(selected_site):
    if selected_site == 'All Sites':
        fig = px.pie(df, names='Launch Site', values='class',
                     title='Total Success Launches for All Sites')
    else:
        filtered_df = df[df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names='class',
                     title=f"Success vs Failure for {selected_site}")
    return fig


@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range
    filtered_df = df[(df['Payload Mass (kg)'] >= low) &
                     (df['Payload Mass (kg)'] <= high)]

    if selected_site != "All Sites":
        filtered_df = filtered_df[filtered_df["Launch Site"] == selected_site]

    fig = px.scatter(filtered_df,
                     x="Payload Mass (kg)",
                     y="class",
                     color="Booster Version Category",
                     title="Correlation between Payload and Success")
    return fig


# ----------------------------
# RUN IN COLAB
# ----------------------------
PORT = 8050

# Function to run the Dash app
def run_dash_app():
    app.run(debug=False, port=PORT)

# Start the Dash app in a separate thread
thread = threading.Thread(target=run_dash_app)
thread.start()

# Give the server a moment to start up
time.sleep(5)

# Serve the kernel port as an iframe
output.serve_kernel_port_as_iframe(PORT)

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8050
INFO:werkzeug:Press CTRL+C to quit


<IPython.core.display.Javascript object>